In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

In [ ]:
from utils import load_data
from port_opt.port_optimizer import PortfolioOptimizer

In [ ]:
# live data from our db
data_all_index = load_data.load_all_data()
# to save data to pkl
# load_data.save_all_data('../data/data_all_index.pkl', data_all_index)
# use pkl for notebook
# data_all_index = load_data.load_all_data_plk('../data/data_all_index.pkl')

In [ ]:
show_data = data_all_index['ibex'][0]
show_data

___

In [ ]:
def get_port_serie(close_data, weights):
    future_data = close_data.loc[:, weights.index]
    future_data = future_data.fillna(method='ffill')
    returns_future_data = future_data.pct_change()
    returns_future_data = returns_future_data + 1
    returns_future_data.iloc[0,:] = weights
    port_data = returns_future_data.cumprod()
    portfolio_data = port_data.sum(axis=1)
    return portfolio_data

def porfolio_from_start_value(portfolio_data, start_value):
    portfolio_data_returns = portfolio_data.pct_change()
    portfolio_data_returns = portfolio_data_returns + 1
    portfolio_data_returns.iloc[0] = start_value
    portfolio_data_start = portfolio_data_returns.cumprod()
    return portfolio_data_start

In [ ]:
def get_portfolios(price_data, max_w=1):
    returns = np.log(price_data).diff()
    returns = returns.dropna()
    po = PortfolioOptimizer(returns)
    
    m_sharpe = po.cla_max_sharpe_portfolio(max_w=max_w)
    m_sharpe.name = 'cla_max_sharpe_portfolio'
    m_vol = po.cla_min_volatility_portfolio(max_w=max_w)
    m_vol.name = 'cla_min_volatility_portfolio'
    ivp = po.ivp_portfolio()
    ivp.name = 'ivp'  
    eq_w = po.eq_w_portfolio()
    eq_w.name = 'eq_w'
    hpr = po.hrp_portfolio()
    hpr.name = 'hrp'
    
    return eq_w, m_sharpe, m_vol, ivp, hpr
    

In [ ]:
from port_opt.hpr import HprAlgorithm


def make_plots_interact(date, n_hist, max_w, sclose, bm, eq_w, ivp, min_vol, max_sharpe, hpr):
    
    date_time = bm.index[date]
    date_time_start_data = bm.index[date-n_hist]
    idx_date = sclose.index.get_loc(date_time)
    
    price_data = sclose.iloc[idx_date-n_hist:idx_date]
    price_data = price_data.dropna(axis=1)
    close_data_future = sclose.iloc[idx_date+1:]
    
    returns = np.log(price_data).diff()
    returns = returns.dropna()
    po = PortfolioOptimizer(returns)
    
    if eq_w:
        eq_w_weights = po.eq_w_portfolio()
        eq_w_weights.name = 'eq_w'
        portfolio_data_eq_w = get_port_serie(close_data_future, eq_w_weights)
        date_start = portfolio_data_eq_w.index[0]
        start_value = bm.loc[date_start]
        portfolio_data_eq_w = porfolio_from_start_value(portfolio_data_eq_w, start_value)
        portfolio_data_eq_w.name='eq_w'

    if ivp:
        ivp_weights = po.ivp_portfolio()
        ivp_weights.name = 'ivp'  
        portfolio_data_ivp = get_port_serie(close_data_future, ivp_weights)
        date_start = portfolio_data_ivp.index[0]
        start_value = bm.loc[date_start]
        portfolio_data_ivp = porfolio_from_start_value(portfolio_data_ivp, start_value)
        portfolio_data_ivp.name = 'ivp'
    
    if max_sharpe:
        m_sharpe_weights = po.cla_max_sharpe_portfolio(max_w=max_w)
        m_sharpe_weights.name = 'cla_max_sharpe_portfolio'
        portfolio_data_max_sharpe = get_port_serie(close_data_future, m_sharpe_weights)
        date_start = portfolio_data_max_sharpe.index[0]
        start_value = bm.loc[date_start]
        portfolio_data_max_sharpe = porfolio_from_start_value(portfolio_data_max_sharpe, start_value)
        portfolio_data_max_sharpe.name = 'max_sharpe'
    
    if min_vol:
        m_vol_weights = po.cla_min_volatility_portfolio(max_w=max_w)
        m_vol_weights.name = 'cla_min_volatility_portfolio'
        portfolio_data_min_vol = get_port_serie(close_data_future, m_vol_weights)
        date_start = portfolio_data_min_vol.index[0]
        start_value = bm.loc[date_start]
        portfolio_data_min_vol = porfolio_from_start_value(portfolio_data_min_vol, start_value)
        portfolio_data_min_vol.name='min_vol'
    
    if hpr:
        hpr_weights = po.hrp_portfolio()
        hpr_weights.name = 'hrp'
        portfolio_data_hrp = get_port_serie(close_data_future, hpr_weights)
        date_start = portfolio_data_hrp.index[0]
        start_value = bm.loc[date_start]
        portfolio_data_hrp = porfolio_from_start_value(portfolio_data_hrp, start_value)
        portfolio_data_hrp.name='hrp'
    
    
    
    fig, ax = plt.subplots(figsize=(25, 7))
    bm.plot(ax=ax)
    
    
    if eq_w:
        portfolio_data_eq_w.plot(ax=ax)
    if ivp:
        portfolio_data_ivp.plot(ax=ax) 
    if max_sharpe:
        portfolio_data_max_sharpe.plot(ax=ax)
    if min_vol:
        portfolio_data_min_vol.plot(ax=ax)
    if hpr:
        portfolio_data_hrp.plot(ax=ax)
    
    
    ax.legend()
    ax.axvline(x=date_time, linewidth=4, color='r')
    ax.axvspan(date_time_start_data, date_time, alpha=0.5, color='grey')
    

    n_len = sum([eq_w, ivp, max_sharpe, min_vol, hpr])
    fig, axes = plt.subplots(1, n_len, figsize=(n_len*5, 5))   
    
    ax_indx = 0
    if eq_w:
        eq_w_weights.plot.pie(ax=axes[ax_indx])
        ax_indx = ax_indx + 1
    if ivp:
        ivp_weights.plot.pie(ax=axes[ax_indx])
        ax_indx = ax_indx + 1
    if max_sharpe:
        m_sharpe_weights.plot.pie(ax=axes[ax_indx])
        ax_indx = ax_indx + 1
    if min_vol:
        m_vol_weights.plot.pie(ax=axes[ax_indx])
        ax_indx = ax_indx + 1
    if hpr:
        hpr_weights.plot.pie(ax=axes[ax_indx])
        ax_indx = ax_indx + 1
        
        
        mu = returns.mean()
        cov = returns.cov()
        corr = returns.corr()
        hpr = HprAlgorithm(cov, corr)
        hpr.plot_dendrogram()
    

In [ ]:
@widgets.interact(
    index=widgets.Dropdown(options=['ibex', 'dax', 'stoxx', 'stoxx_sectors'], value='ibex', description='index:'),
    date=widgets.IntSlider(min=250, max=data_all_index['stoxx_sectors'][2], step=10, value=1000, layout=widgets.Layout(width='50%')),
    max_w=widgets.FloatSlider(min=0.05, max=1, step=0.05, value=1.0, layout=widgets.Layout(width='50%')),
    eq_w=widgets.Checkbox(value=True, description='eq_w', disabled=False),
    ivp=widgets.Checkbox(value=True, description='ivp', disabled=False),
    min_vol=widgets.Checkbox(value=False, description='min_vol', disabled=False),
    max_sharpe=widgets.Checkbox(value=False, description='max_sharpe', disabled=False),
    hpr=widgets.Checkbox(value=False, description='hpr', disabled=False),
    n_hist=widgets.IntSlider(min=10, max=250*5, step=10, value=250, layout=widgets.Layout(width='50%')),
)
def plot(index, date, max_w, n_hist, eq_w, ivp, min_vol, max_sharpe, hpr):
    sclose, bm, n_days = data_all_index[index]
    make_plots_interact(date, n_hist, max_w, sclose, bm, eq_w, ivp, min_vol, max_sharpe, hpr)